In [1]:
import re
import os
import json
import pandas as pd

In [2]:
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())

In [3]:
persony = pd.read_json(os.path.join("data", "persony_raw.json"))

In [4]:
persony

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
0,Ondřej Mašek (1000),1000,,fyzická,None,None,None,None,None,None
1,kino Světozor (100019),100019,None,právní,None,None,None,None,None,None
2,"AD WOOD, s.r.o. (100038)",100038,None,právní,None,None,None,None,None,None
3,Hypermarket Film s.r.o. (100039),100039,None,právní,Hypermarket Film,None,None,None,None,None
4,Hypermarket Film s.r.o. (100039),100039,None,právní,Hypermarket Film,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
35810,Josef Dvořák (9998),9998,"herec, divadelní podnikatel",fyzická,Josef Voloďa Dvořák (rodné jméno),None,25.04.1942,"Horní Cerekev, Československo",None,None
35811,Antonín Dimitrov (99996),99996,"výtvarník, grafik, scénograf, designér",fyzická,None,None,27.02.1928,"Mšecké Žehrovice, Československo",27.12.2014,"Bobcaygeon, Kanada"
35812,Čechomor (person),person,None,právní,None,None,None,None,None,None
35813,Kevin Mc Kidd (person),person,,fyzická,None,None,None,None,None,None


In [5]:
def odnebovat(x):
    if x:
        if "nebo" in x:
            x = x.split("nebo")[1].strip()
    return x

In [6]:
def rozdel_misto(x):
    if type(x) == str:
        try:
            mesto = x.split(",")[0]
            stat = x.split(",")[1]
        except:
            mesto = None
            stat = x
    else:
        mesto = None
        stat = None
    return [mesto, stat]

## Doplnění dat z Wikidat

In [7]:
persony[persony["Jméno"] == "Stanislav Litera (35280)"].sample(1)

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18963,Stanislav Litera (35280),35280,,fyzická,None,None,None,None,None,None


In [8]:
with open(os.path.join("data_fixes", "data_z_wikidat.json"), encoding="utf-8") as data_z_wikidat:
    data_z_wikidat = json.loads(data_z_wikidat.read())

In [9]:
for key, val in data_z_wikidat.items():
    persony.loc[persony["Jméno"] == key, "Datum narození"] = val[0]

In [10]:
for key, val in data_z_wikidat.items():
    persony.loc[persony["Jméno"] == key, "Datum úmrtí"] = val[1]

In [11]:
persony[persony["Jméno"] == "Stanislav Litera (35280)"].sample(1)

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18963,Stanislav Litera (35280),35280,,fyzická,None,None,11.06.1931,None,21.09.2003,None


## Opravy

In [12]:
persony[persony["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18566,Eva Pecková (34563),34563,"pedagožka, zpěvačka, tanečnice",fyzická,Eva Kubičková (rodné jméno),None,09.12.9441,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika"


In [13]:
for key, val in opravy["data_narozeni"].items():
    persony.loc[persony["Jméno"] == key, "Datum narození"] = val

In [14]:
for key, val in opravy["jmena"].items():
    persony.loc[persony["Jméno"] == key, "Jméno"] = val

In [15]:
persony[persony["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18566,Eva Pecková (34563),34563,"pedagožka, zpěvačka, tanečnice",fyzická,Eva Kubičková (rodné jméno),None,,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika"


In [16]:
persony[persony["Jméno"] == "Adam Klemens (1779)"]

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
11585,Adam Klemens (1779),1779,"dirigent, hudební skladatel",fyzická,None,None,14.01.1967,None,None,None


## Konverze formátů

In [17]:
persony["Datum narození"].sample(200).unique()

array(['10.05.1760', '13.02.1945', None, '14.04.1925', '23.01.1911',
       '22.08.1943', '08.01.1974', '25.10.1947', '1829', '1944',
       '23.04.1962', '10.09.1965', '25.03.1878', '08.07.1904',
       '04.05.1973', '18.10.1984', '16.09.1982', '28.04.1915',
       '14.06.1941', '29.09.1965', '17.08.1983', '12.10.1913',
       '19.05.1954', '01.05.1941', '1935', '27.11.1923', '30.11.1954',
       '31.03.1962', '22.10.1927', '17.05.1955', '10.07.1889',
       '01.08.1848', '04.07.1980', '1928', '14.07.1910', '25.05.1903',
       '20.06.1926', '10.04.1975', '14.07.1939', '20.11.1961', '1939',
       '22.05.1925', '1963', '17.12.1971', '29.05.1975', '29.03.1976',
       '19.09.1912', '07.07.1910', '08.08.1931', '22.09.1987',
       '12.12.1954', '25.11.1993', '08.04.1933', '03.04.1947',
       '28.11.1925', '24.02.2005', '21.09.1923', '1966', '28.05.1906',
       '28.12.1971', '23.02.1972', '21.02.1973', '26.05.1921',
       '14.05.1902', '1987', '22.07.1993', '05.05.1906', '06.12.1953']

In [18]:
persony["Město narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[0])
persony["Stát narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[1])
persony["Město úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Stát úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Rok narození"] = pd.to_numeric(persony["Datum narození"].apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")
persony["Rok úmrtí"] = pd.to_numeric(persony["Datum úmrtí"].apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")

In [19]:
def na_data(sloupec):
    global persony
    persony["Pracovni"] = persony[sloupec]
    persony["Pracovni"] = persony["Pracovni"].apply(lambda x: odnebovat(x))
    for index, row in persony.iterrows():
        try:
            date = pd.to_datetime(row["Pracovni"], format = "%d.%m.%Y", errors="ignore")
        except ValueError:
            date = pd.to_datetime(row["Pracovni"], format = "%Y", errors="ignore")
        except ValueError:
            date = None
        persony.at[index, sloupec] = date
    persony = persony.drop(["Pracovni"], axis = 1)

In [20]:
na_data("Datum narození")
na_data("Datum úmrtí")

In [21]:
persony = persony.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [22]:
persony

,Jméno,Id,Medailon,Osoba,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Město narození,Stát narození,Město úmrtí,Stát úmrtí,Rok narození,Rok úmrtí
0,Ondřej Mašek (1000),1000,,fyzická,None,None,None,None,None,None,None,None,None,None,NaN,NaN
1,kino Světozor (100019),100019,None,právní,None,None,None,None,None,None,None,None,None,None,NaN,NaN
2,"AD WOOD, s.r.o. (100038)",100038,None,právní,None,None,None,None,None,None,None,None,None,None,NaN,NaN
3,Hypermarket Film s.r.o. (100039),100039,None,právní,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN
4,Hypermarket Film s.r.o. (100039),100039,None,právní,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35810,Josef Dvořák (9998),9998,"herec, divadelní podnikatel",fyzická,Josef Voloďa Dvořák (rodné jméno),None,1942-04-25 00:00:00,"Horní Cerekev, Československo",None,None,Horní Cerekev,Československo,None,None,1942.0,NaN
35811,Antonín Dimitrov (99996),99996,"výtvarník, grafik, scénograf, designér",fyzická,None,None,1928-02-27 00:00:00,"Mšecké Žehrovice, Československo",2014-12-27 00:00:00,"Bobcaygeon, Kanada",Mšecké Žehrovice,Československo,Kanada,Kanada,1928.0,2014.0
35812,Čechomor (person),person,None,právní,None,None,None,None,None,None,None,None,None,None,NaN,NaN
35813,Kevin Mc Kidd (person),person,,fyzická,None,None,None,None,None,None,None,None,None,None,NaN,NaN


In [23]:
persony.to_json(os.path.join("data", "persony.json"), orient="records")